# Analyse en Composantes Principales sur des transcriptomes de patients atteints de diabète de type 1
------------------------------------------------------------------------------------------------------


Dans ce TP vous allez réaliser une Analyse en Composantes Principales (ACP). Cette méthode permet d'évaluer la variabilité de vos données. Elle repose sur une transformation des variables (ici: niveaux de transcrits) partiellement corrélées entre elles en de nouvelles variables dé-corrélées les unes des autres. L'ACP vous permet ainsi d'optimiser la représentation de données massives pour en tirer un maximum d'information.

Vous allez analyser des données de transcriptome générées sur puces d'expression, les mêmes que celles étudiées au cours des TP 6 et 7 de l'UE1.

__Les ARN ont été extraits à partir de cultures de lignées lymphoblastoïdes (lymphocytes B immortalisés par le virus EBV) de patients et apparentés (germains partageant les mêmes allèles HLA), stimulés 6h ou 24h avec du Phorbol Myristate Acetate (PMA), ou non stimulés.__

Vous allez :
- Réaliser l'ACP en utilisant le package FactoMineR de R.
- Analyser les graphiques des individus et repérer si les nouveaux axes séparent des groupes d'individus pertinents sur le plan clinique
- Analyser les cercles des corrélations et identifier les transcrits les mieux représentés par les axes de l'ACP.




---
---

## 1. Préparation de l'environnement R

Travaillez dans un repertoire `PCA` dans lequel vous mettrez ce notebook.

Choix du répertoire de travail et chargement du paquet FactoMineR (déjà installé)

In [ ]:
#cell 1
getwd()
list.files() # on verifie que l'objet for_factominer.RData est bien présent sinon on décommente la ligne suivante:
#setwd('~/m1_meg-acp')

In [ ]:
#cell 2
# Importation du package FactoMineR:
options(width=160)
installed.packages()["FactoMineR",c(1,2,3)] # pour vérifier que le paquet est bien installé

if (!require("FactoMineR", quietly = T)) {
    install.packages("FactoMineR")}  # commande pour installer factominer au cas où -> peut-être utile si vous travailler sur un autre ordinateur/serveur

library(FactoMineR)


In [ ]:
#cell 3
cat("Voici mon environnement de travail avec les paquets de R chargés:\n")
sessionInfo()

Importation des données de transcriptome:

In [ ]:
#cell 4
load("for_factominer.RData")
ls()

L'objet for_factominer contient les variables en colonnes, ainsi que des variables supplémentaires, qualitatives et quantitatives.

- Combien ce dataframe contient-elle de lignes et de colonnes ? A quoi correspondent-elles ?
Faites attention notamment aux 6 dernières colonnes du dataframe.

*__Fonctions à utiliser__*: `str()` ou `dim()` ou `nrow()` et `ncol()`; `head()` et `tail()`

In [ ]:
# cell 5

N'affichez la structure que des 10 dernières colonnes:

In [ ]:
#cell 6
str(for_factominer[,(ncol(for_factominer)-9):ncol(for_factominer)])

> Vos commentaires :

Autre façon de visualiser le dataframe:
-> ici en le transposant
-> en utilisant la fonction datatable de la librairie "DT" qui permet de naviguer avec les ascenceurs dans la table d'un notebook et de filtrer

In [ ]:
#cell 7
if (!require("DT", quietly = T)) {
    install.packages("DT")}  
options(warn = -1)
transposed_data <- as.data.frame(t(for_factominer), stringsAsFactors=F)
str(transposed_data)
transposed_data$Gene_name <- row.names(transposed_data)
transposed_data <- transposed_data[,c(ncol(transposed_data), 1: ncol(transposed_data)-1)]
datatable(transposed_data, rownames=F, caption="Dataset for ACP analysis", filter="top", options = list(lengthMenu=c(10,20, 30, 40, 50)))

---

## 2. Analyse en Composantes Principales par FactoMineR

### 2.1. Création des résultats de l'ACP:

La fonction `PCA()` de FactoMineR crée un objet de type `list` contenant tous les résultats de l'ACP.

In [ ]:
#cell 8
# Utilisation de la fonction PCA() de FactoMineR
#?PCA
PCAres <- PCA(for_factominer, quali.sup = 1921:1925, quanti.sup = 1926, graph = FALSE)
#str(PCAres) # longue structure!
print(class(PCAres))
names(PCAres)

### 2.2 Analyse visuelle des deux premières composantes de l'ACP
Les deux premières composantes rendent comptent au maximum de l'inertie du nuage de points d'origine. Visualisons les résultats grâce à un graphique affichant la position des individus en fonction des deux premières composantes principes.

In [ ]:
#cell 9
# Création de graphiques indiquant la position des échantillons par rapport aux deux premières composantes et enregistrement dans un fichier PDF:
#pdf("PCA_individus.pdf")

plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", cex=0.3)
plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", habillage = "Stim", cex=0.3)
plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", cex=0.3, habillage = "PedID")
plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", cex=0.3, habillage = "ID")
plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", cex=0.3, habillage = "Status")
plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", cex=0.3, habillage = "Sex")

#dev.off()

> Pensez à ajouter un titre et une légende!

Figure 1. Titre
Légende

__Commentez les éléments communs à ces différents graphiques, notamment  :__
- à quoi correspondent les axes de ces graphiques ?
- que signifient les pourcentages associés à chacun de ces graphiques ?
- que représentent les points de ce graphique et les coordonnées de ces points ?
- les points son-ils étalés sur ces deux axes ? Constate-t-on des regroupements de certains points ?

__Commentez ensuite pour chacun de ce ces graphiques :__
- Constatez-vous un lien entre les annotations indiquées en couleurs de ces points et leur position sur chacun des axes créés par l'ACP ?

Vos commentaires :

> Eléments communs à tous les graphiques :

> Eléments spécifiques à chaque graphique :

### 2.3. Qualité de l'ACP

Plus les axes de l'ACP rendent comptent de l'inertie du nuage de points d'origine, meilleure est la qualité de l'ACP (cela signifie que l'on a maintenu au mieux la forme du nuage de points d'origine).
L'inertie exprimée par chacune des composantes est enregistrée dans `PCAres$eig`.

In [ ]:
#cell 10
# Inertie traduite par les 5 premières composantes
PCAres$eig[1:5]

__Commentez ces valeurs :__
- Quel pourcentage de l'inertie du nuage de point d'origine est traduit par les composantes 1 et 2 ? 
- Quel pourcentage de l'intertie est traduit par les cinq premières composantes ?

> Vos commentaires :

Il est possible d'afficher sous forme graphique le pourcentage d'inertie traduit par chaque dimension :

In [ ]:
# cell 11
# Graphique Inertia and dimensions

#pdf("PCA_inertia.pdf")
eig.val <- PCAres$eig
barplot(eig.val[, 2], 
        names.arg = 1:nrow(eig.val), 
        main = "Variances Explained by PCs (%)",
        xlab = "Principal Components",
        ylab = "Percentage of variances",
        ylim = c(0,15),
        col ="steelblue")

barplot(eig.val[, 3], 
        names.arg = 1:nrow(eig.val), 
        main = "Variances Explained by PCs (%)",
        xlab = "Principal Components",
        ylab = "Cumulative Percentage of variance",
        ylim = c(0,100),
        col ="steelblue")

#dev.off()

__Comment évolue l'inertie avec les différentes composantes ?__
- Combien de composantes sont finalement nécessaires pour rendre compte de 50% de l'inertie du nuage d'origine ?
- Comment évolue le pourcentage d'inertie traduit par les composantes ?

> Vos commentaires :

---

## 3. analyse de corrélations entre les variables du dataframe et/ou de l'ACP

### 3.1. Corrélation entre les différentes composantes et les variables supplémentaires (traitement, sexe, âge, famille, individu)

Plutôt que d'étudier de façon visuelle chacune des composantes, il est possible de tester la corrélation entre les différentes composantes et chacune des variables. On peut ainsi mettre en évidence que les échantillons situés sur les valeurs basses de la composante 1 auront par exemple été traitées avec le PMA, tandis que les échantillons situés dans les valeurs hautes n'auront pas été stimulés.

In [ ]:
#cell 12
# Test de corrélation entre les variables supplémentaires qualitatives et les composantes

round(PCAres$quali.sup$eta2,2)

In [ ]:
#cell 13
# Test de corrélation entre les variables supplémentaires quantitatives et les composantes

round(PCAres$quanti.sup$cor,2)

__Commentez les résultats de ces tests de corrélation__
- Quelle(s) composante(s) vous semble corrélée(s) avec quelle(s) variable(s) supplémentaire(s) ?
- Cela va-t-il dans le même sens que vos observations sur les deux premières composantes ?

> Vos commentaires :

*__Vous pouvez observer les éventuelles corrélations entre variables qualitatives et composantes via des graphiques !__*

En particulier, utilisez l'argument `axes` pour spécifier les dimensions d'intérêt comme dans l'exemple ci-dessous.

In [ ]:
#cell 14
#pdf("nom_fichier.pdf")
# les composantes par défaut sont les 1 et 2
# vous pouvez les modifier via l'argument "axes" (cf. exemple ci-dessous pour les composantes3 et 4)

plot(PCAres, choix = "ind", autoLab = "yes", invisible = "quali", cex=0.3, habillage = "Status", axes = c(3,4))

#dev.off()

> Vos commentaires :

### 3.2 Corrélation entre les différentes composantes et les variables utilisées par l'ACP (quantité des ARNm)

Un gène ayant fortement contribué à une composante sera corrélé avec elle ! 

Cela signifie que :
- les individus situés dans les valeurs hautes de la composante expriment fortement ce gène (corrélation, 0 < coeff. de corrélation < 1)

- OU les individus situés dans les valeurs hautes de la composante expriment faiblement ce gène (corrélation inverse, -1 < coeff. de corrélation < 0)
 
 Le __cercle de corrélation__ révèle la corrélation entre les différentes variables (ici les niveaux de transcrits) et les composantes. Chaque transcrit est représenté par une flèche. La position de l'extrémité de la flèche indique le coefficient de corrélation entre le transcrit et les deux composantes affichées.

In [ ]:
#cell 15
# Cercle de corrélation
# L'argument "contrib 30" indique que vous choisisser de n'afficher que les 30 transcrits les plus corrélés avec les composantes 1 et 2

#pdf(file = "PCA_diab_contrib30.pdf")
plot(PCAres, choix = "var", cex = 0.6, select = "contrib 30", unselect = 1)
#dev.off()

__Identifiez quelques transcrits dont les niveaux vous semblent corrélés avec les composantes 1 et 2__
- Identifiez les transcrits représenrés par des flèches proches de l'un des axes
- S'agit-il d'une corrélation ou d'une corrélation inverse ?

> Vos commentaires :

Vous pouvez calculer le niveau de corrélation de ces transcrits avec la position de ces échantillons sur les axes qui vous intéressent.

In [ ]:
#cell 16
# Lignes de codes pour calculer les coefficients de corrélation entre le niveau des transcrits et la position des échantillons sur les axes
cor.test(for_factominer$SLAMF1,PCAres$ind$coord[,1])
cor.test(for_factominer$SLAMF1,PCAres$ind$coord[,2])
cor.test(for_factominer$RGS16,PCAres$ind$coord[,1])
cor.test(for_factominer$RGS16,PCAres$ind$coord[,2])

> Vos commentaires : (lien entre la flèche du cercle de corrélation et les valeurs que vous venez de calculer)

__Cette analyse a-t-elle répondu à votre question scientifique ? Quelles analyses pourriez vous imaginer afin d'aller plus loin ?__

---
---
# Fil Rouge RNASeq final part option 2:

<span style="color:red">Au choix, vous rendrez ce travail portant sur l'ACP des données de puces d'expression ou le Fil Rouge final part option 1 portant sur l'analyse différentielle du RNASeq.</span>

## Consignes pour votre compte-rendu de TP ACP

Vous allez réaliser un compte-rendu récapitulant les informations importantes ce TP. Les consignes sont les suivantes :
    
- Compte-rendu à remettre sur Moodle pour le **3 décembre 2021 23h59** 
- Un compte-rendu par binôme
- formats possibles: `.docx`, `.pdf` ou un notebook `.ipynb` ou `.html`. 
- Nombre limite de caractères : 4000 en dehors des légendes des figures pour les formats `docx` et `pdf`
(4000 caractères en taille 12 Times New Roman en interligne simple représentent un texte un peu aéré d'environ une page et demi -> ajustez au mieux pour la version notebook)

Le compte-rendu inclura : 
- un **titre** pertinent décrivant le message essentiel de votre compte-rendu ("Compte-rendu de TP ACP" ne constitue donc pas un titre pour ce 
travail)

- une **brève introduction** situant votre analyse dans les recherches sur le diabète. Elle amène à la question posée par cette étude(2 pts avec 
le titre)

- un chapitre **Résutats** qui décrit les résultats obtenus et concluent de façon factuelle ce que vous avez découvert via l'ACP. Ils seront 
illustrés par une/des figure(s), que vous devez citer dans le texte de façon pertinente. Les figures doivent être légendées. La légende inclut le titre de la figure et doit être suffisamment explicite, en quelques lignes, pour que votre lecteur soit capable de la comprendre (et de se forger sa propre opinion) sans avoir à lire votre texte. (4 pts texte + 2 pts figures)

- une **brève discussion/conclusion** dans laquelle vous pourrez donner:
    1) une interprétation des résultats allant au delà d'une pure description, par exemple en indiquant si vos résultats ont apporté des éléments nouveaux sur le diabète
    2) des idées d'expériences possibles pour aller au-delà
    3) une phrase d'ouverture.
    
Dans certains articles la discussion peut contenir un schéma de modèle (mais cela ne vous est pas demandé pour ce TP). (2 pts)

***Rappel important :***<br>
Les plagiats sont interdits. Des comptes-rendus considérés comme excessivement proches par le correcteur pourront entraîner des 
procédures disciplinaires.
